In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-06"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8708,2024-05-06,Brasil Nbb,19:00,Paulistano,Franca,2.06,1.69,152.5,1.87,1.79,1.5,1.93,1.55,WpxU7PiL,0.485437,0.591716,0.534759,0.558659,0.077153,0.6,0.6,NaN,NaN,145.584,31.309398,0.215061,103.700,15.141745,0.146015,117.262,128.210,104.00,98.55,0.0,0.0,0.0,0.0,0.139536,0.030912,0.154426,-1.06,-0.212,-5.000000,0.665288,0.5,-0.165288,0.61,0.122,5.655738,0.713419,0.8,0.086581
8709,2024-05-06,Eua Nba,23:00,Denver Nuggets,Minnesota Timberwolves,1.45,2.87,205.5,1.80,1.95,-6.5,2.00,4.75,0QskIaNp,0.689655,0.348432,0.555556,0.512821,0.038087,0.2,0.6,NaN,NaN,172.276,28.639645,0.166243,223.916,55.781146,0.249116,156.582,222.632,141.48,220.82,0.0,0.0,0.0,0.0,0.464857,0.056569,0.576161,0.76,0.152,2.960526,0.735161,0.9,0.164839,1.07,0.214,8.738318,0.567771,0.7,0.132229
8710,2024-05-06,Croácia Premijer Liga,14:30,Zabok,Cibona,1.76,1.97,161.5,1.87,1.79,-2.5,2.02,2.38,YkbsPZ76,0.568182,0.507614,0.534759,0.558659,0.075796,0.6,0.4,NaN,NaN,183.268,171.231444,0.934323,147.670,38.992946,0.264055,239.354,245.658,489.28,154.23,1.0,0.0,0.0,0.0,0.079621,0.030912,0.115708,1.74,0.348,2.183908,0.000000,0.0,0.000000,9.60,1.920,0.505208,0.000000,0.0,0.000000
8711,2024-05-06,Croácia Premijer Liga,15:45,Dinamo Zagreb,Cedevita,1.66,2.13,169.5,1.85,1.83,-2.5,1.86,2.48,MBvxoHUP,0.602410,0.469484,0.540541,0.546448,0.071893,0.4,0.8,NaN,NaN,206.478,119.505158,0.578779,130.782,19.433505,0.148595,162.498,122.686,201.14,120.65,0.0,0.0,0.0,0.0,0.175377,0.007686,0.202031,-3.75,-0.750,-0.880000,0.000000,0.0,0.000000,1.75,0.350,3.228571,0.000000,0.0,0.000000
8712,2024-05-06,Hungria Nacional,13:00,Szolnoki Olaj,Alba Fehervar,1.72,2.02,177.5,1.85,1.81,-2.5,1.98,2.43,pjePRHGN,0.581395,0.495050,0.540541,0.552486,0.076445,0.8,0.8,NaN,NaN,163.090,23.163618,0.142030,163.896,65.315601,0.398519,142.288,163.896,191.58,135.20,0.0,0.0,0.0,0.0,0.113440,0.015456,0.144308,-0.45,-0.090,-8.000000,0.000000,0.0,0.000000,-0.65,-0.130,-7.846154,0.622627,0.5,-0.122627
8713,2024-05-06,Islândia Premier League Feminina,16:15,Grindavik F,Njardvik F,1.72,2.02,150.5,1.85,1.85,-2.5,1.98,2.43,E3GQpJRr,0.581395,0.495050,0.540541,0.540541,0.076445,0.8,0.6,NaN,NaN,134.338,27.752623,0.206588,94.114,11.577676,0.123018,112.728,101.302,122.01,109.98,0.0,0.0,0.0,0.0,0.113440,0.000000,0.144308,0.06,0.012,60.000000,0.000000,0.0,0.000000,-0.87,-0.174,-5.862069,0.000000,0.0,0.000000
8714,2024-05-06,Israel Liga Leumit,14:15,Maccabi Raanana,Ironi Nahariya,1.91,1.81,169.5,1.81,1.87,-1.5,1.85,1.92,EHowNZI8,0.523560,0.552486,0.552486,0.534759,0.076046,0.8,0.2,NaN,NaN,139.756,21.850479,0.156347,153.518,44.827486,0.292001,141.004,156.950,153.85,148.00,0.0,0.0,0.0,0.0,0.038016,0.023058,0.026259,0.03,0.006,151.666667,0.000000,0.0,0.000000,1.07,0.214,3.785047,0.000000,0.0,0.000000
8715,2024-05-06,Japão B.League,02:05,Kyoto,Nagasaki,2.18,1.64,162.5,1.87,1.79,1.5,2.01,1.50,dtoq7Pz0,0.458716,0.609756,0.534759,0.558659,0.068472,0.6,0.6,NaN,NaN,188.804,40.144901,0.212627,194.694,143.255272,0.735797,170.868,275.368,143.00,132.84,0.0,0.0,0.0,0.0,0.199915,0.030912,0.205484,-5.00,-1.000,-1.180000,0.373256,0.2,-0.173256,-1.64,-0.328,-1.951220,0.428994,0.4,-0.028994


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,23:00,Eua Nba,Denver Nuggets,Minnesota Timberwolves,205.5,1.80,1.0,Over
1,14:30,Croácia Premijer Liga,Zabok,Cibona,161.5,1.87,1.0,Over
2,15:45,Croácia Premijer Liga,Dinamo Zagreb,Cedevita,169.5,1.85,1.0,Over
3,13:00,Hungria Nacional,Szolnoki Olaj,Alba Fehervar,177.5,1.85,1.0,Over
4,16:15,Islândia Premier League Feminina,Grindavik F,Njardvik F,150.5,1.85,1.0,Over
